In [1]:
'''
# Merge the follow data packages into consolidated data packages based on assay type.

+ KINOMEscan kinase small molecule binding assay
+ KiNativ kinase small molecule binding assay
+ Fluorescence imaging apoptosis assay
+ Fluorescence imaging cell growth inhibition assay

'''
__author__ = 'Zichen Wang (zichen.wang@mssm.edu)'

import sys
reload(sys)  
sys.setdefaultencoding('utf8')
# print sys.getdefaultencoding()

import os, tarfile, codecs
from collections import OrderedDict
import pandas as pd

def read_manifest(fn):
	# Read Manifest file
	df = pd.read_csv(fn, sep='\t')
	df.set_index('datasetid', inplace=True)
	df['metadata_contents'] = df['metadata_contents'].map(lambda x: x.split('\t'))
	return df

def read_data(fn):
	if fn.endswith('txt'):
		sep = '\t'
	elif fn.endswith('.csv'):
		sep = ','
	df = pd.read_csv(fn, sep=sep) 
	return df

def read_readme(fn):
	out = OrderedDict()
	with codecs.open(fn, 'r', 'utf-8') as f:
		is_section = False
		subsection = ''
		lines = [line for line in f]
		section_idx = set()
		for i, line in enumerate(lines):

			if i not in section_idx:
				if '####' in line:
					is_section = not is_section
					if is_section:
						section = lines[i+1].strip()
						out[section] = OrderedDict()
						out[section][subsection] = ''
						section_idx.add(i+1)
						# print section

				elif line.strip().endswith(':'):
					subsection = line.strip()
					# print subsection
					out[section][subsection] = ''
				else:
					out[section][subsection] += line
	# print out['Dataset Information']['Dataset Contents:']
	return out

def write_readme(d, fn):
	with open(fn, 'w') as out:
		for section in d:
			out.write('#' * 64 + '\n')
			out.write(section + '\n')
			out.write('#' * 64 + '\n')
			out.write('\n')
			for subsection in d[section]:
				out.write(subsection + '\n')
				out.write(d[section][subsection])
	return

def merge_readme(meta1, meta2):
	meta = OrderedDict()
	for section in meta1.keys():
		if meta1[section] == meta2[section]:
			meta[section] = meta1[section]
		else:
			meta[section] = OrderedDict()
			for subsection in meta1[section].keys():
				try:
					if meta1[section][subsection] == meta2[section][subsection]:
						meta[section][subsection] = meta1[section][subsection]
					else:
						meta[section][subsection] = ''
				except KeyError:
					pass
	return meta

def extract_tgz(fn):
	# To extract a .tar.gz
	dirname = fn.split('.')[0]
	tf = tarfile.open(fn, 'r')
	tf.extractall()
	return os.listdir(dirname)


In [2]:
def _merge(manifest_fn, dataset_ids_to_exclude=[]):
	manifest = read_manifest(manifest_fn)
	## drop unwanted rows in manifest
	manifest = manifest.drop(dataset_ids_to_exclude, axis=0)

	# print manifest.head()

	# Get unique metadata filenames
	unique_metadata_fns = set(reduce(lambda x,y: x+y, manifest['metadata_contents']))

	merged_metadata = dict(zip(unique_metadata_fns, 
		[pd.DataFrame()]*len(unique_metadata_fns))) # to collect merged metadata dfs

	merged_data_df = pd.DataFrame() # to collect merged data
	merged_readme = OrderedDict()

	for dataset_id in manifest.index:
		tgz_fn = '%s.tar.gz' % dataset_id 
		filenames = extract_tgz(tgz_fn)
		# Read and merge metadata
		metadata_fns = manifest.ix[dataset_id]['metadata_contents']
		# print metadata_fns
		for metadata_fn in metadata_fns:
			meta_df = read_data('%s/%s' % (dataset_id, metadata_fn))
			merged_metadata[metadata_fn] = merged_metadata[metadata_fn].append(meta_df)

		# Get dataset filename
		dataset_fn = filter(lambda x: x not in unique_metadata_fns, filenames)
		dataset_fn = filter(lambda x: 'readme' not in x.lower(), dataset_fn)
		# print dataset_fn
		print dataset_id # , filenames

		assert len(dataset_fn) == 1
		dataset_fn = dataset_fn[0]
		# Read and merge data
		df = read_data('%s/%s' % (dataset_id, dataset_fn))
		print df.columns
		merged_data_df = merged_data_df.append(df)
		# print 'Merged %s' % dataset_fn 
		# print df.shape, merged_data_df.shape

		# Read ReadMe.txt
		try:
			readme = read_readme('%s/ReadMe.txt' % dataset_id)
		except UnicodeDecodeError:
			pass
		else:
			if len(merged_readme) == 0:
				merged_readme = readme
			else:
				merged_readme = merge_readme(merged_readme, readme)

	# Drop duplicated rows
	# print merged_data_df.shape
	merged_data_df = merged_data_df.drop_duplicates()
	print merged_data_df.shape
	return merged_data_df

In [3]:
# KiNativ
merged_data_df = _merge('Manifest_1458120819559.txt', ['LDS-1258', 'LDS-1259', 'LDS-1260'])
merged_data_df.head()

,datarecordID,hmsDatasetID,smCenterCompoundID,smSalt,smCenterSampleID,smLincsID,smName,clName,clCenterSpecificID,ppName,ppLincsID,recordedPlate,recordedWell,controlType,datapointName,datapointUnit,datapointValue
0,71929,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200633,NaN,NaN,NaN,sequence,NaN,LMTGDTYTAHAGAkFPIK
1,71929,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200633,NaN,NaN,NaN,labelingSite,NaN,Activation Loop
2,71929,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200633,NaN,NaN,NaN,percentInhibition,NaN,2.7
3,71929,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200633,NaN,NaN,NaN,assayCompoundConcentration,NaN,10
4,71929,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200633,NaN,NaN,NaN,concUnit,NaN,uM


In [4]:
merged_data_df.shape

(26404, 17)

In [5]:
merged_data_df.count()

datarecordID          26404
hmsDatasetID          26404
smCenterCompoundID    26404
smSalt                26404
smCenterSampleID      26404
smLincsID             26404
smName                26404
clName                26404
clCenterSpecificID    26404
ppName                26404
ppLincsID             26404
recordedPlate             0
recordedWell              0
controlType               0
datapointName         26404
datapointUnit          1401
datapointValue        26307
dtype: int64

In [6]:
# drop columns with 0 data
cols_to_drop = merged_data_df.columns[merged_data_df.count() == 0]
merged_data_df = merged_data_df.drop(cols_to_drop, axis=1)
merged_data_df.shape

(26404, 14)

In [7]:
# groupby datarecordID and get the first rows in each group except for datapointValue and datapointName
grouped_df_meta = merged_data_df.groupby('datarecordID').head(1)\
    .drop(['datapointName', 'datapointValue'],axis=1)\
    .set_index('datarecordID')
grouped_df_meta.head()

,hmsDatasetID,smCenterCompoundID,smSalt,smCenterSampleID,smLincsID,smName,clName,clCenterSpecificID,ppName,ppLincsID,datapointUnit
datarecordID,,,,,,,,,,,
71929,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200633,NaN
71930,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"ABL,ARG",200634,NaN
71931,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,ACK,200635,NaN
71932,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,AGK,200636,NaN
71933,20095,10080,101,1,LSM-1080,Torin2,HeLa,50061,"AMPKa1,AMPKa2",200639,NaN


In [8]:
grouped_df_meta.shape

(5561, 11)

In [9]:
# pivot_table to get datapointName on the columns
grouped_df = pd.pivot_table(merged_data_df[['datarecordID', 'datapointName', 'datapointValue']],
                            values='datapointValue', 
                            index=['datarecordID'], 
                            columns=['datapointName'],
                            aggfunc=lambda x: x,
                            dropna=False)

grouped_df.head()

,IC50,assayCompoundConcentration,concUnit,labelingSite,percentInhibition,sequence
datarecordID,,,,,,
70118,>10,NaN,uM,Activation Loop,NaN,LMTGDTYTAHAGAkFPIK
70119,>10,NaN,uM,LYS1,NaN,YSLTVAVkTLKEDTMEVEEFLK
70120,>10,NaN,uM,LYS1,NaN,TVSVAVKCLkPDVLSQPEAMDDFIR
70121,>10,NaN,uM,ATP,NaN,ATVFLNPAACkGK
70122,>10,NaN,uM,ATP Loop,NaN,GTFGkVILVK


In [10]:
grouped_df.shape

(5561, 6)

In [11]:
grouped_df.count()

IC50                          1394
assayCompoundConcentration    4160
concUnit                      5561
labelingSite                  5543
percentInhibition             4088
sequence                      5561
dtype: int64

In [12]:
# join grouped_df and grouped_df_meta on datarecordID
grouped_df = grouped_df.merge(grouped_df_meta, left_index=True, right_index=True, how='inner')
# export this 
grouped_df.to_csv('KiNativ_kinase_small_molecule_binding_assay/HMS_LINCS-KiNativ_kinase_small_molecule_binding_assay.csv')
grouped_df.shape

(5561, 17)

In [13]:
## create experimentID to uniquely identify each experiment
grouped_df['experimentID'] = grouped_df[[
        'hmsDatasetID',
        'clCenterSpecificID',        
        'smCenterCompoundID',        
        'assayCompoundConcentration']].apply(lambda x: '-'.join(map(str, x)), axis=1)


In [14]:
# check dtypes
grouped_df.dtypes

IC50                          object
assayCompoundConcentration    object
concUnit                      object
labelingSite                  object
percentInhibition             object
sequence                      object
hmsDatasetID                   int64
smCenterCompoundID             int64
smSalt                         int64
smCenterSampleID               int64
smLincsID                     object
smName                        object
clName                        object
clCenterSpecificID             int64
ppName                        object
ppLincsID                      int64
datapointUnit                 object
experimentID                  object
dtype: object

In [15]:
# separate row meta (kinases) and column meta (experiments)
row_meta_df = grouped_df.reset_index()[['labelingSite', 'sequence', 'ppName', 'ppLincsID']].drop_duplicates()
col_meta_df = grouped_df.reset_index()[['experimentID', 'assayCompoundConcentration', 'concUnit', 
                          'hmsDatasetID', 'smCenterCompoundID', 'smSalt',
                         'smCenterSampleID', 'smLincsID', 'smName',
                         'clName', 'clCenterSpecificID']].drop_duplicates().set_index('experimentID').sort_index()
row_meta_df.head()

,labelingSite,sequence,ppName,ppLincsID
0,Activation Loop,LMTGDTYTAHAGAkFPIK,"ABL,ARG",200633
1,Lys1,YSLTVAVkTLKEDTMEVEEFLK,"ABL,ARG",200634
2,Lys1,TVSVAVkCLKPDVLSQPEAMDDFIR,ACK,200635
3,ATP,ATVFLNPAACkGK,AGK,200636
4,Lys2,DLkPENVLLDAHMNAK,"AMPKa1,AMPKa2",200639


In [16]:
row_meta_df.shape

(1045, 4)

In [17]:
row_meta_df.apply(lambda x: x.nunique())

labelingSite     20
sequence        748
ppName          325
ppLincsID       494
dtype: int64

In [18]:
row_meta_df['labelingSite'].unique()

array(['Activation Loop', 'Lys1', 'ATP', 'Lys2', 'ATP Loop',
       'Protein Kinase Domain', 'Other', nan, 'LYS1', 'LYS2',
       'Kinase Domain', 'ACT', 'K1', 'LIP', 'K2', 'LIP (PI3K/PI4K)',
       'LIP (PI5K)', 'nonKD (autoinhibition domain)', 'KD',
       'Other (MAPKAP2/3 active site in p38a complex)', 'nonKD'], dtype=object)

In [19]:
## clean up row_meta_df by combining different labelingSite namings
row_meta_df.groupby(['sequence'])['labelingSite'].agg(lambda x:','.join(map(str,x))).head(20)

sequence
AACLLDGVPVALKK                         K1,Lys1
AACLLDGVPVALkK                       Lys1,LYS1
AAGIGKDFKENPNLR                            nan
AAGIGkDFKENPNLR                      Other,ATP
ADNTLVAVkSCR                              Lys1
AIHKETGQIVAIkQVPVESDLQEIIK           Lys1,LYS1
AIQFLHQDSPSLIHGDIKSSNVLLDER            K2,Lys2
AIQFLHQDSPSLIHGDIkSSNVLLDER          Lys2,LYS2
AKDLPTFKDNDFLNEGQK              LIP (PI5K),ATP
AKDLPTFkDNDFLNEGQK                         ATP
AKELPTLKDNDFINEGQK              LIP (PI5K),ATP
AKELPTLkDNDFINEGQK                         ATP
ALYATKTLR                              K1,Lys1
ALYATkTLR                            Lys1,LYS1
APVAIKVFK                              K1,Lys1
APVAIkVFK                            Lys1,LYS1
AQNKETSVLAAAkVIDTK                   Lys1,LYS1
AQTHGkWPVKWYAPECINYYK          Activation Loop
ARDTVTSELAAVkIVK                          Lys1
ATFSFCVSPLLVFVNSKSGDNQGVK                  ATP
Name: labelingSite, dtype: object

In [20]:
## Assume ppLincsID, drop the sequence and labelingSite
# row_meta_df = row_meta_df[['ppLincsID','ppName']].drop_duplicates().set_index('ppLincsID')
row_meta_df = grouped_df.reset_index()[['ppName', 'ppLincsID']].drop_duplicates().set_index('ppLincsID').sort_index()
row_meta_df.shape

(494, 1)

In [21]:
row_meta_df.head()

,ppName
ppLincsID,
200633,"ABL,ARG"
200634,"ABL,ARG"
200635,ACK
200636,AGK
200637,AKT1


In [22]:
col_meta_df.apply(lambda x: x.nunique())

assayCompoundConcentration     5
concUnit                       1
hmsDatasetID                  24
smCenterCompoundID            24
smSalt                         2
smCenterSampleID               1
smLincsID                     24
smName                        24
clName                         7
clCenterSpecificID             7
dtype: int64

In [23]:
col_meta_df.shape

(34, 10)

In [24]:
col_meta_df.head()


,assayCompoundConcentration,concUnit,hmsDatasetID,smCenterCompoundID,smSalt,smCenterSampleID,smLincsID,smName,clName,clCenterSpecificID
experimentID,,,,,,,,,,
20087-50060-10008-nan,NaN,uM,20087,10008,101,1,LSM-1008,Sorafenib,A-375,50060
20088-50060-10029-nan,NaN,uM,20088,10029,101,1,LSM-1029,GW-5074,A-375,50060
20089-50060-10046-nan,NaN,uM,20089,10046,101,1,LSM-1046,SB590885,A-375,50060
20090-50060-10049-nan,NaN,uM,20090,10049,101,1,LSM-1049,PLX-4720,A-375,50060
20091-50060-10050-nan,NaN,uM,20091,10050,101,1,LSM-1050,AZ-628,A-375,50060


In [25]:
# pivot_table to make a matrix of percentInhibition
value_matrix = pd.pivot_table(grouped_df.reset_index()[['percentInhibition', 'ppLincsID', 'experimentID']].drop_duplicates(),
                            values='percentInhibition', 
                            columns='experimentID',
                            index='ppLincsID', 
                            aggfunc=lambda x: x,
                            dropna=False)

value_matrix.head()

,20087-50060-10008-nan,20088-50060-10029-nan,20089-50060-10046-nan,20090-50060-10049-nan,20091-50060-10050-nan,20092-50060-10068-nan,20093-50060-10017-10,20094-50061-10079-10,20095-50061-10080-10,20097-50061-10086-10,...,20104-50454-10200-0.5,20104-50454-10200-5,20105-50061-10201-10,20106-50061-10070-10,20106-50454-10070-10,20204-51097-10075-1,20206-50811-10129-1,20207-51099-10188-1,20209-50335-10337-1,20210-50811-10351-1
ppLincsID,,,,,,,,,,,,,,,,,,,,,
200633,NaN,NaN,NaN,NaN,NaN,NaN,96.2,-1.9,2.7,-2.8,...,39.9,80.6,-2.6,-31.1,NaN,25.73794561,-12.2,-62.8,95.0,>90
200634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.0,-4.2,NaN,...,NaN,NaN,7.8,NaN,NaN,NaN,NaN,-49.5,NaN,NaN
200635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.6,4.1,69.0,...,0.3,25.4,94.0,-1.0,NaN,NaN,-39.5,-6.4,72.3,-33.4
200636,NaN,NaN,NaN,NaN,NaN,NaN,~NI,-0.8,20.1,NaN,...,2.2,-6.6,-2.5,NaN,-1.8,NaN,NaN,-23.6,NaN,NaN
200637,NaN,NaN,NaN,NaN,NaN,NaN,3.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
value_matrix.shape

(494, 34)

In [27]:
grouped_df['percentInhibition'].count()

4088

In [28]:
value_matrix.count().sum()

4085

In [29]:
## make gct object
sys.path.append('/Users/zichen/Documents/GitHub/l1ktools/python')
import cmap.io.gct as gct
g = gct.GCT()
print value_matrix.shape
print row_meta_df.shape
print col_meta_df.shape
g.build_from_DataFrame(value_matrix, rdesc=row_meta_df, cdesc=col_meta_df)


In [30]:
## export to gct and gctx
g.write('KiNativ_kinase_small_molecule_binding_assay/HMS_LINCS-KiNativ_kinase_small_molecule_binding_assay'
        , mode='gct')
# g.write('HMS_LINCS-KiNativ_kinase_small_molecule_binding_assay.', mode='gctx')